In [1]:
import os
!wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/02.02-Asynchrony/Sources/ach.h -nv -O ach.h
!wget https://raw.githubusercontent.com/NVIDIA/accelerated-computing-hub/refs/heads/main/tutorials/cuda-cpp/notebooks/02.02-Asynchrony/Sources/nvtx3.hpp -nv -O nvtx3.hpp
!wget https://github.com/giovadg/physnum_CUDA/tree/main/waves_pde/ConfigFile.h -nv -O ConfigFile.h
!wget https://github.com/giovadg/physnum_CUDA/tree/main/waves_pde/ConfigFile.hpp -nv -O ConfigFile.hpp


'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.
'wget' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub > /dev/null 2>&1
!sudo add-apt-repository -y "deb https://developer.download.nvidia.com/devtools/repos/ubuntu$(source /etc/lsb-release; echo "$DISTRIB_RELEASE" | tr -d .)/$(dpkg --print-architecture)/ /" > /dev/null 2>&1
# !sudo apt install -y nsight-systems > /dev/null 2>&1
# !sudo apt update -qq
# !sudo apt install -y nsight-systems-cli

The system cannot find the path specified.
The system cannot find the path specified.


In [ ]:
%%writefile script.cpp
#include <iostream>
#include <vector>
#include <fstream>
#include <cmath>
#include <numeric>
#include <cstdlib>
#include <cstdio>
#include <algorithm>
#include <chrono>
#include <string.h>
#include "ConfigFile.h"
#include "ach.h"
#include "nvtx3.hpp"


using namespace std;


struct func_ic{
  int N_ic_thet;
  double dth, dth_dot;

  __host__ __device__ thrust::tuple<double,double> operator () (int ii) {
    int jj = ii / N_ic_thet;
    int kk = ii % N_ic_thet;
    double th_dot = dth_dot * jj;
    double th     = dth     * kk;
    return thrust::make_tuple(th, th_dot);
  }
}

class Exercice2
{

private:
  double t, dt, tFin;
  double m, g, L;
  double Omega, kappa;
  double theta0_l, theta0_r, thetadot0_l, thetadot0_r;
  double B0, B1;
  double mu;
  double om_0, om_1, nu, Ig;
  thrust::universal_vector<double> theta_vect, thetadot_vect;

  int N_excit, nsteps_per, Nperiod, N_ic_thet, N_ic_thetdot, N_ic;
  int sampling;
  int last;
  ofstream *outputFile;

  void printOut(bool force)
  {
    if((!force && last>=sampling) || (force && last!=1))
    {
      double emec = 0.5*Ig*pow(thetadot,2) - mu*B0*cos(theta); // TODO: Evaluer l'energie mecanique
      double pnc = thetadot*( -kappa*thetadot - mu*B1*sin(theta)*sin(Omega*t)); // TODO: Evaluer la puissance des forces non conservatives

      *outputFile << t << " " << theta_vect[0] << " " << thetadot_vect[0] << " " << emec << " " << pnc << endl;
      last = 1;
    }
    else
    {
      last++;
    }
  };

    // TODO define acceleration functions and separate contribution 
    // for a[0] = a(x,t), a[1] = a(v,t)
  __host__ __device__ thrust::universal_vector<double> acceleration(double x, double v, double t_)
  {
    thrust::universal_vector<double> acc(2);
    acc[0] = -pow(om_0,2)*sin(x) -pow(om_1,2)*sin(Omega*t_)*sin(x); // acceleration depending on x only
    acc[1] =  -nu*v; // acceleration depending on v only

    return acc;
  }

__host__ __device__  thrust::tuple<double,double> step(theta, thetadot)
  {
    thrust::universal_vector<double> a = acceleration(theta, thetadot, t);

    double old_theta = theta;
    double old_thetadot = thetadot;
    double t_p1;
    double zvhalf;

    theta        = old_theta + old_thetadot*dt + (a[0]+a[1])* pow(dt,2) /2.0;
    zvhalf       = old_thetadot + (a[0]+a[1])*dt/2;
    thrust::universal_vector<double> a_phalf  = acceleration(old_theta, zvhalf,t+dt/2);
    thrust::universal_vector<double> a_p1    = acceleration(theta, old_thetadot, t+dt);
    thetadot     = old_thetadot + (a[0]+a_p1[0])*dt/2 + a_phalf[1]*dt;

    return thrust::make_tuple(theta, thetadot);
  }

struct func_run{
  Exercice2* obj;

  __device__ __host__ thrust::tuple<double,double> operator() (const thrust::tuple<double,double>& zip){
    double theta    = thrust::get<0>(zip);
    double thetadot = thrust::get<1>(zip);

    return obj->step(theta, thetadot);
  }
}




public:
  Exercice2(int argc, char* argv[])
  {
    const double pi=3.1415926535897932384626433832795028841971e0;
    string inputPath("configuration.in"); // Fichier d'input par defaut
    if(argc>1) // Fichier d'input specifie par l'utilisateur ("./Exercice2 config_perso.in")
      inputPath = argv[1];

    ConfigFile configFile(inputPath); // Les parametres sont lus et stockes dans une "map" de strings.

    for(int i(2); i<argc; ++i) // Input complementaires ("./Exercice2 config_perso.in input_scan=[valeur]")
      configFile.process(argv[i]);

    //tFin     = configFile.get<double>("tFin");      // t final (overwritten if N_excit >0)
  //  dt       = configFile.get<double>("dt");        // time step (overwritten if nsteps_per >0)
    Omega    = configFile.get<double>("Omega");     // frequency magnetic field
    kappa    = configFile.get<double>("kappa");     // coefficient for friction
    m        = configFile.get<double>("m");         // mass
    L        = configFile.get<double>("L");         // length
    B1       = configFile.get<double>("B1");     //  B1 part of magnetic fields (oscillating part amplitude)
    B0       = configFile.get<double>("B0");     //  B0 part of magnetic fields (static part amplitude)
    mu       = configFile.get<double>("mu");     //  moment magnetique 
    theta0_l    = configFile.get<double>("theta0_l");    // initial condition in theta left
    theta0_r    = configFile.get<double>("theta0_r");    // initial condition in theta right
    thetadot0_l = configFile.get<double>("thetadot0_l"); // initial condition in thetadot left
    thetadot0_r = configFile.get<double>("thetadot0_r"); // initial condition in thetadot right
    N_ic_thet       = configFile.get<double>("N_ic_thet");        // Number of initial condition
    N_ic_thetdot       = configFile.get<double>("N_ic_thetdot");        // Number of initial condition
    sampling   = configFile.get<int>("sampling");     // number of time steps between two writings on file
    N_excit    = configFile.get<int>("N_excit");            // number of periods of excitation
    Nperiod    = configFile.get<int>("Nperiod");      // number of periods of oscillation theoritique
    nsteps_per= configFile.get<int>("nsteps");      // number of time step per period

    // Ouverture du fichier de sortie
    outputFile = new ofstream(configFile.get<string>("output").c_str());
    outputFile->precision(15);

    Ig   = (1./12.)*m*pow(L,2);
    om_0 =  sqrt(mu*B0/Ig);
    om_1 = sqrt(mu*B1/Ig);
    nu   = kappa/Ig;

    N_ic = N_ic_thetdot * N_ic_thet;
    dth     = (theta0_r - theta0_l)/N_ic_thet;
    dth_dot = (thetadot0_r - thetadot0_l)/N_ic_thetdot;

    theta_vect.resize(N_ic);
    thetadot_vect.resize(N_ic);
  
    auto zip_begin = thrust::make_zip_iterator(thrust::make_tuple(theta_vect, thetadot_vect));
    auto zip_end   = zip_begin + N_ic;

    auto fun_ic = func_ic{N_ic_thet, dth, dth_dot};
    thurst::tabulate(thrust::host, zip_begin, zip_end, fun_ic) //TODO


    if(N_excit>0){
      tFin = N_excit*(2*pi/Omega);
      dt   = (2*pi/Omega)/nsteps_per;
    }
    else{
      tFin = Nperiod * (2.*pi/om_0);
      dt   = (2*pi/om_0)/nsteps_per;
    } 
    cout << "final time is "<<"  "<< tFin << endl; 
    cout << "omega_0 is " << om_0 <<endl;
    cout << "period is " << (2.*pi/om_0) <<endl;
  }
  

  ~Exercice2()
  {
    outputFile->close();
    delete outputFile;
  };

    void run()
  {
    t = 0.;
    last = 0;
    printOut(true);

    auto zip_begin = thrust::make_zip_iterator(thrust::make_tuple(theta_vect.begin(), thetadot_vect.begin()));

    auto zip_end = zip_begin + N_ic;

    while (t < tFin - 0.5 * dt)
    {
      func_run time_step_fun{this,t};
      thrust::transform(thrust::host,zip_begin, zip_end, zip_begin, time_step_fun);

      t += dt;
      printOut(false);
    }
    printOut(true);

  };

};

int main(int argc, char* argv[])
{
  Exercice2 engine(argc, argv);
  engine.run();

  return 0;
}



In [ ]:
!nvcc -o /tmp/b_gpu.out --extended-lambda script.cpp -x cu -arch=native # build executable

In [ ]:
!nsys profile --cuda-event-trace=false --force-overwrite true -o script_profiled /tmp/b_gpu.out input # run and profile executable
